### 금융정보방 게시판에 있는 글들 다 모아오기.
#### author : 편도훈



In [1]:
from selenium import webdriver
import selenium
import pandas as pd
import numpy as np
import datetime
import time
from tqdm import tqdm
from glob import glob
import datetime
import os
from multiprocessing.pool import ThreadPool

In [2]:
# 1개 회사의 정보와 폴더이름을 가지고 게시판에 들어가서 크롤링하는 function입니다.

def crawl_(company_info_lists,folder_name):

    # 크롬드라이버가 연결되어 있는 주소 저장
    chromedriver ='E:/big12/python-project/note/driver/chromedriver.exe'

    # 크롬드라이버 연결
    driver = webdriver.Chrome(chromedriver)
    print(company_info_lists)
    #업종 사이트 연결
    title=[]
    times=[]
    contents=[]
    address=[]
    #받아온 정보의 index 0 인 코드를 넣습니다. page는 아무렇게나 하셔도 상관 없습니다. but 저는 1번째 페이지만 해놨습니다.
    driver.get('https://finance.naver.com/item/board.nhn?code={code}&page={page}'.format(code=company_info_lists[0],page=1))

    table = driver.find_element_by_class_name('type2')

    #게시글 제목 찾기
    title.extend([x.text.replace(","," ") for x in table.find_elements_by_tag_name('a')]) 

    #게시글 링크 찾기
    address.extend([x.get_attribute('href') for x in table.find_elements_by_tag_name('a')])

    #게시글 작성 시간 찾기
    times.extend([x.text for x in table.find_elements_by_class_name('tah.p10.gray03') if len(x.text)>10])

    #게시글 내용 찾기

    for addr in address:
        driver.get(addr)
        contents.append(driver.find_element_by_id('body').text.replace(","," "))

    now= datetime.datetime.now().strftime("%Y_%m_%d_%H_%M_%S")
    board_contents = pd.DataFrame(
        {'title':title,'contents':contents,'time':times}
    )
    board_contents.to_csv('./categories/'+folder_name+'/'+company_info_lists[1]+now+'.csv', encoding='utf-8')
    driver.close()


In [3]:
# #파일경로
# file_paths= glob('E:/big12/python-project/note/categories/*.csv')
# file_path_root ='E:/big12/python-project/note/categories/'

def folder_file_separator(file_paths):
    
    #업종별 csv가 저장되어있는 경로를 입력해주세요(루트까지만)
    file_path_root ='E:/big12/python-project/note/categories/'
#파일이름을 기초로 찾아보기.
    for file_path in file_paths:
        #종목 폴더이름
        folder_name=[]
        folder_name = str(file_path.split('\\')[1].split('.')[0])
        try:
            if not(os.path.isdir(file_path_root+folder_name)):
                os.makedirs(os.path.join(file_path_root+folder_name))
        except OSError as e:
            if e.errno != errno.EEXIST:
                print("Failed to create directory!!!!!")
                raise
        time.sleep(0.7)

        #종목 회사 별코드
        code_list= [x[1:] for x in np.array(pd.read_csv(file_path)['code'])]
        #종목별회사명
        name_list = [x for x in np.array(pd.read_csv(file_path)['name'])]
        company_info_list=[]
        for code,name in zip(code_list,name_list):
            company_info_list.append([code,name])

        for company_infos in company_info_list:
            print(company_infos,folder_name,'from separator')
            crawl_(company_infos,folder_name)

In [ ]:
#파일경로
###4개의 멀티쓰레딩으로 정보를 모을겁니다. 
### 코어의 개수 * 2 하신게 최대 쓰레드라고 보시면 되고요.. 저는 그냥 반정도 욕심내지 않고 하려합니다.
### 멀티쓰레딩에 사용되는 모듈 및 function은 from multiprocessing.pool import ThreadPool입니다.
### author : 편도훈 a.k.a '중원구 민트초코 패밀리사이즈' 입니다.

## 업종별 모음하신 파일 경로를 입력해주세요.
file_paths_all= glob('E:/big12/python-project/note/categories/*.csv')

file_paths1=[]
file_paths2=[]
file_paths3=[]
file_paths4=[]
for i in range(len(file_paths_all)):
    if i%4==0: file_paths1.append(file_paths_all[i])
    elif i%4==1 : file_paths2.append(file_paths_all[i])
    elif i%4==2 : file_paths3.append(file_paths_all[i])
    elif i%4==3 : file_paths4.append(file_paths_all[i])
### starmap은 왠지 모르겠는데 2중리스트를 1중리스트로 풀어서 그 1중리스트를 또 풀어 인자로 전달한다.. .인자만 16개가 되는 대 참 사가
### 일어나기 때문에 한번 더 감싸줘서 3중리스트로 안전하고 꼼꼼하게 우겨넣었더니 되더라
file_paths_separated_list= [[file_paths1],[file_paths2],[file_paths3],[file_paths4]]
with ThreadPool(4) as tp:
    tp.starmap(
    folder_file_separator,file_paths_separated_list
    )

['024940', 'PN풍년']['003480', '한진중공업홀딩스'] 가스유틸리티 from separator
 가정용기기와용품 from separator
['011090', '에넥스'] 가구 from separator
['050960', '수산아이앤티'] IT서비스 from separator
['024940', 'PN풍년']['011090', '에넥스']

['050960', '수산아이앤티']
['003480', '한진중공업홀딩스']
['016710', '대성홀딩스'] 가스유틸리티 from separator
['039290', '인포뱅크']['134790', '시디즈']['192400', '쿠쿠홀딩스'] IT서비스 from separator
  가구 from separator
가정용기기와용품 from separator
['016710', '대성홀딩스']
['134790', '시디즈']
['039290', '인포뱅크']
['192400', '쿠쿠홀딩스']
['307950', '현대오토에버'] IT서비스 from separator
['017390', '서울가스'] 가스유틸리티 from separator
['017390', '서울가스']
['307950', '현대오토에버']
['009240', '한샘'] 가구 from separator
['256150', '한독크린텍'] 가정용기기와용품 from separator
['009240', '한샘']
['256150', '한독크린텍']
['348030', '모비릭스'] IT서비스 from separator
['348030', '모비릭스']
['117580', '대성에너지'] 가스유틸리티 from separator
['013890', '지누스'] 가구 from separator
['117580', '대성에너지']
['037070', '파세코'] 가정용기기와용품 from separator
['037070', '파세코']
['013890', '지누스']
['060250', 'NHN한국사이버결제'] IT서비스 from sepa

In [ ]:
# #파일경로
###개갓다
# file_paths= glob('E:/big12/python-project/note/categories/*.csv')
# file_path_root ='E:/big12/python-project/note/categories/'
# #파일이름을 기초로 찾아보기.
# for file_path in file_paths[1:]:
#     #종목 폴더이름
#     folder_name = str(file_path.split('\\')[1].split('.')[0])
    
#     try:
#         if not(os.path.isdir(file_path_root+folder_name)):
#             os.makedirs(os.path.join(file_path_root+folder_name))
#     except OSError as e:
#         if e.errno != errno.EEXIST:
#             print("Failed to create directory!!!!!")
#             raise
#     time.sleep(0.7)
    
#     #종목 회사 별코드
#     code_list= [x[1:] for x in np.array(pd.read_csv(file_path)['code'])]
#     #종목별회사명
#     name_list = [x for x in np.array(pd.read_csv(file_path)['name'])]
#     company_info_list=[]
#     for code,name in zip(code_list,name_list):
#         company_info_list.append([code,name])
        
#     for company_infos in company_info_list:
#         print(company_infos,folder_name)
#         crawl_(company_infos,folder_name)